In [1]:
import requests
import json
import nltk
!pip install stop-words
from stop_words import get_stop_words
import io
import os
!pip install wget
import wget
import shutil
from tensorflow import keras
import zipfile
import numpy as np
import pandas as pd
import string
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, RocCurveDisplay, plot_roc_curve, f1_score
import re
import spacy
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import TextVectorization
import tensorflow as tf

fast_run = input('\n\nCzy włączyć szybkie uruchomienie? Jeśli tak tweety zostaną wczytane z pliku, a nie pobrane z API. Odpowiedź tak / nie: ').lower()

if not (fast_run == 'tak' or fast_run == 'nie'):
    raise TypeError('odpowiedź powinna brzmieć tak / nie')

fast_run = True if fast_run == 'tak' else False

if not fast_run: # test
    print('Przy pobieraniu z API w lokalizacji notatnika powinien znaleźc się plik token.txt z bearer tokenem dostępowym do API')
    with open('token.txt', 'r') as file:
        token = file.read()

    headers = {'Authorization': f'Bearer {token}'}
    response = requests.get('https://api.twitter.com/2/tweets/1221826367928655876', headers=headers)
    print(response.json()) 



Czy włączyć szybkie uruchomienie? Jeśli tak tweety zostaną wczytane z pliku, a nie pobrane z API. Odpowiedź tak / nie: nie
Przy pobieraniu z API w lokalizacji notatnika powinien znaleźc się plik token.txt z bearer tokenem dostępowym do API
{'data': {'id': '1221826367928655876', 'text': 'MSZ przygotowuje ewakuację Polaków z chińskiego Wuhan https://t.co/Mk9D4bS8Ob #wieszwiecej'}}


### API blokuje mnie za każdym razem po pobraniu 25 - 30 tysięcy tweetów z informacją że limit został przekroczony. Dlatego pobieram tyle, na ile limit pozwala. Nie ma to jednak wpływu na jakośc zadania - wyniki i tak są dobre

In [17]:
def request_tweets_from_api():
    url = 'https://raw.githubusercontent.com/PatrycyD/ZUM_NLP/master/pl_covid_tweets_clean.txt'
    wget.download(url) if not os.path.isfile('pl_covid_tweets_clean.txt') else 0

    i = 0
    with open('pl_covid_tweets_clean.txt', 'r') as file:
        tweets_ids = [line.split('\t')[0] for line in file]

    tweets_ids = tweets_ids[1:] #pominięcie nagłówka
    tweets_ids = tweets_ids[:50000]

    print(f'Liczba przygotowanych id tweetow: {len(tweets_ids)}')

    tweets = {}
    i = 0
    for tweet_ids_list in range(0, len(tweets_ids), 100):
        i = i + 100
        group_ids = ','.join(tweets_ids[tweet_ids_list:tweet_ids_list+100])
        r = requests.get(f'https://api.twitter.com/2/tweets?ids={group_ids}', headers=headers)
        if r.status_code == 429:
          print('Limit reached. Quiting loop')
          break
          # raise OverflowError('API requests limit reached. Pleas try again in a few minutes')
        try:
            for tweet in dict(r.json())['data']:
                tweets[tweet['id']] = tweet['text']
        except KeyError: #gdy wszystkie pobrany teety nie istnieją, nie ma klucza 'data'
            continue

        
        if i % 5000 == 0:
            print(f'{i} tweets ids processed')

    print(f'\nLiczba pobranych tweetów: {len(tweets)}')
    
    with open('/content/drive/MyDrive/tweets.json', 'w', encoding='utf-8') as file:
        file.write(json.dumps(tweets))
    
    return tweets

def load_tweets_from_repo():
    url = 'https://raw.githubusercontent.com/PatrycyD/ZUM_NLP/master/tweets.json'
    wget.download(url) if not os.path.isfile('tweets.json') else 0
    file = open('tweets.json', 'r', encoding="utf-8")
    tweets = file.read()
    tweets = tweets.replace("'", '"')
    tweets = json.loads(tweets)
    
    return tweets
    
if fast_run:
    tweets = load_tweets_from_repo()
else:
    tweets = request_tweets_from_api()


Liczba przygotowanych id tweetow: 50000
5000 tweets ids processed
10000 tweets ids processed
15000 tweets ids processed
20000 tweets ids processed
25000 tweets ids processed
30000 tweets ids processed
Limit reached. Quiting loop

Liczba pobranych tweetów: 22135


In [18]:
def cleaning_URLs(data):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ',data)

punctuations_list = string.punctuation
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list) # jest to mapowanie i zamiana, dwa pierwsze argumenty to dwa stringi, gdzie znaki w pierwszym są zamieniane na znaki w drugim stringu, zgodnie z indeksem, Trzeci argument to znaki, które są mapowane do None => zostaną po prostu usunięte
    return text.translate(translator)

def cleaning_repeating_char(text):
    return re.sub(r'(.)1+', r'1', text)

def clean_numbers(data):
    return re.sub('[0-9]+', '',
                  data)

!pip install --upgrade spacy
!python -m spacy download pl_core_news_sm
nlp = spacy.load('pl_core_news_sm')
def lemmatize(text):
    doc = nlp(text)
    lemmatized_sentence = ''
    for token in doc:
        lemmatized_sentence = f'{lemmatized_sentence} {token.lemma_}'
        
    return lemmatized_sentence.strip()

stop_words = get_stop_words('polish')
no_stop_words_tweets = {key: ' '.join([word for word in value.split() if word 
                                       not in stop_words]) for key, value in tweets.items()}

tweets_no_urls = {k: cleaning_URLs(v) for k, v in no_stop_words_tweets.items()}
tweets_no_punct = {k: cleaning_punctuations(v) for k, v in no_stop_words_tweets.items()}
tweets_no_repeating_chars = {k: cleaning_repeating_char(v) for k, v in tweets_no_punct.items()}
tweets_no_numbers = {k: clean_numbers(v) for k, v in tweets_no_urls.items()}
tweets_lemmatized = {k: lemmatize(v) for k, v in tweets_no_numbers.items()}

     |████████████████████████████████| 58.6 MB 1.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_sm')


In [19]:
embedding_file = 'pl-embeddings-cbow.txt'

if not os.path.isfile(embedding_file):
    url = 'http://publications.ics.p.lodz.pl/2016/word_embeddings/pl-embeddings-cbow.txt'
    wget.download(url)
    
embeddings_index = {}
with open(embedding_file, encoding='utf-8') as file:
    for line in file:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs
        
embeddings_index.pop('933198') #na początku pliku jest notatka o liczbie wektorów i liczbie punktów w wektorach

print("Found %s word vectors." % len(embeddings_index))

Found 933198 word vectors.


In [20]:
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = [v for k, v in tweets_lemmatized.items()]
vectorizer.adapt(text_ds)

output = vectorizer([['szczepienie choroba wirus chory zdrowy koronawirus']])
output.numpy()[0, :6]

array([2599,   92,   11,   62,  531,    8])

In [21]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

test = ['szczepienie', 'choroba', 'wirus', 'chory', 'zdrowy', 'koronawirus']
[word_index[w] for w in test]

[2599, 92, 11, 62, 531, 8]

In [22]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, idx in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[idx] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 14876 words (5124 misses)


In [23]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(embedding_matrix)
classes = kmeans.predict(embedding_matrix)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(embedding_matrix, classes, test_size=0.1)

rf = RandomForestClassifier(n_estimators=15, max_depth=5)
knn = KNeighborsClassifier(n_neighbors=20)
svc = SVC()

rf.fit(X_train, y_train)
knn.fit(X_train, y_train)
svc.fit(X_train, y_train)

rf_preds = rf.predict(X_test)
knn_preds = knn.predict(X_test)
svc_preds = svc.predict(X_test)

rf_train_preds = rf.predict(X_train)
knn_train_preds = knn.predict(X_train)
svc_train_preds = svc.predict(X_train)

In [25]:
print('Random Forest test predictions')
print(confusion_matrix(y_test, rf_preds))
print('\n\nK Nearest Neighbors test predictions')
print(confusion_matrix(y_test, knn_preds))
print('\n\nSupport Vector Machines test predictions')
print(confusion_matrix(y_test, svc_preds))

Random Forest test predictions
[[1237    0   32]
 [  24  160   13]
 [ 166    3  366]]


K Nearest Neighbors test predictions
[[1222    0   47]
 [  10  186    1]
 [ 163    1  371]]


Support Vector Machines test predictions
[[1254    3   12]
 [   1  196    0]
 [  12    1  522]]


In [26]:
print('Random Forest train predictions')
print(confusion_matrix(y_train, rf_train_preds))
print('\n\nK Nearest Neighbors train predictions')
print(confusion_matrix(y_train, knn_train_preds))
print('\n\nSupport Vector Machines train predictions')
print(confusion_matrix(y_train, svc_train_preds))

Random Forest train predictions
[[11397     3   256]
 [  162  1440    76]
 [ 1460     7  3200]]


K Nearest Neighbors train predictions
[[11281     4   371]
 [   51  1622     5]
 [ 1312     3  3352]]


Support Vector Machines train predictions
[[11650     1     5]
 [    0  1678     0]
 [    3     1  4663]]


### Nie mogę użyć ROC do ewaluacji modelu, ponieważ klasyfikacja jest niebinarna, użyję F1

In [27]:
print(f'Ranom Forest test f1: {round(f1_score(y_test, rf_preds, average="macro"), 2)}')
print(f'Ranom Forest train f1: {round(f1_score(y_train, rf_train_preds, average="macro"), 2)}')

print(f'\nK Nearest Neighbors test f1: {round(f1_score(y_test, knn_preds, average="macro"), 2)}')
print(f'K Nearest Neighbors train f1: {round(f1_score(y_train, knn_train_preds, average="macro"), 2)}')

print(f'\nSupport Vector Machines test f1: {round(f1_score(y_test, svc_preds, average="macro"), 2)}')
print(f'Support Vector Machines train f1: {round(f1_score(y_train, svc_train_preds, average="macro"), 2)}')

Ranom Forest test f1: 0.86
Ranom Forest train f1: 0.88

K Nearest Neighbors test f1: 0.89
K Nearest Neighbors train f1: 0.9

Support Vector Machines test f1: 0.98
Support Vector Machines train f1: 1.0


### Wyniki są dobre

In [28]:
#sprzątanie:
!rm pl-embeddings-cbow.txt
!rm pl_covid_tweets_clean.txt
![[ -f tweets.json ]] && rm tweets.json